## Step 1: Import all libraries


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
from PIL import Image

## Step 2: Download data

In [10]:
!pip3 install gdown 
!gdown https://drive.google.com/uc?id=1TWjI7ucryVPGMpUMq8cdigYrtKQtc54t
!unzip kagglecatsanddogs_3376a.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1TWjI7ucryVPGMpUMq8cdigYrtKQtc54t
From (redirected): https://drive.google.com/uc?id=1TWjI7ucryVPGMpUMq8cdigYrtKQtc54t&confirm=t&uuid=aa7bc705-d49c-4976-9c62-3261fc15b0c0
To: C:\Users\Asus\kagglecatsanddogs_3367a.zip

  0%|          | 0.00/825M [00:00<?, ?B/s]
  0%|          | 1.05M/825M [00:00<02:25, 5.65MB/s]
  0%|          | 2.62M/825M [00:00<01:34, 8.67MB/s]
  0%|          | 3.67M/825M [00:00<01:31, 9.00MB/s]
  1%|          | 4.72M/825M [00:00<01:29, 9.16MB/s]
  1%|          | 5.77M/825M [00:00<01:28, 9.26MB/s]
  1%|          | 6.82M/825M [00:00<01:27, 9.36MB/s]
  1%|          | 7.86M/825M [00:00<01:26, 9.44MB/s]
  1%|1         | 8.91M/825M [00:00<01:26, 9.42MB/s]
  1%|1         | 9.96M/825M [00:01<01:25, 9.51MB/s]
  1%|1         | 11.0M/825M [00:01<01:25, 9.48MB/s]
  1%|1         | 12.1M/825M [00:01<01:25, 9.52MB/s]
  2%|1         | 13.1M/825M [00:01<01:25, 9.48MB/s]
  2%|1         | 14.7M/825M [00:01<01:20, 10.0MB/s]
 

## Step 3: Clean data and remove non image files


In [ ]:
#cleaning data

path = 'PetImages'

for folder in os.listdir(path): #Get all folder names --> ['Cat', 'Dog']
    for img_file in os.listdir(os.path.join(path, folder)): #Loop each folder to get all image files
        img_file = os.path.join(path, folder, img_file) #creating fill path for each image file
        
        try:
            img = Image.open(img_file)
            if img.mode != 'RGB':
                os.remove(img_file) #removing gray scale images
            except:
                os.remoce(img_file) #removing file tyoe None images

In [ ]:
## Step 4: Data preprocessing and convert to tensor format

In [ ]:
transform = transforms.Compose([
                                transforms.Resize(255),  # resize img to 255px square image
                                transforms.CenterCrop(224), # then center crop by 224px
                                transforms.ToTensor(), # convert into tensor format
                                transforms.Normalize([0.5], [0.5]) # normalize tensors.
                               ]) 

dataset = datasets.ImageFolder('PetImages', transform=transform)

dataset_len = len(dataset) # get total number of images in all folders.

train_len, test_len = dataset_len-6000, 6000 # Keep 6000 images for test_set and others are for training
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len]) # split train and test dataset
batch_size = 200

# train and test dataloader
train_set = DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size)
test_set = DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)

# if cuda available then use device as cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Using device: ',device)

## Step 5: Build Model

In [ ]:
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    
    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(p=0.2)
    
    self.conv1 = nn.Conv2d( in_channels=3,  out_channels=6,  kernel_size=4)
    self.conv2 = nn.Conv2d( in_channels=6,  out_channels=12, kernel_size=4)
    self.conv3 = nn.Conv2d( in_channels=12, out_channels=14, kernel_size=4)
    self.conv4 = nn.Conv2d( in_channels=14, out_channels=16, kernel_size=4)
    self.conv5 = nn.Conv2d( in_channels=16, out_channels=20, kernel_size=4)
    
    self.fc1 = nn.Linear( in_features= 20*4*4, out_features=250 ) 
    self.fc2 = nn.Linear( in_features=250,     out_features=200 )
    self.fc3 = nn.Linear( in_features=200,     out_features=50  )
    self.fc4 = nn.Linear( in_features=50,      out_features=10  )
    self.fc5 = nn.Linear( in_features=10,      out_features=2   )
  
  
  def forward(self, x): # sahpe of x is [batch_size, channel, height, width]
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = self.pool(F.relu(self.conv4(x)))
    x = self.pool(F.relu(self.conv5(x)))

    x = x.reshape(-1, 20*4*4)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = self.dropout(F.relu(self.fc4(x)))
    x = self.fc5(x)
    return x


net = Model().to(device)

print(net)

## Step 6: Define Loss and optimizer functions

In [ ]:
# optimizer and loss functions
criterion = nn.CrossEntropyLoss() # crossentropy loss for classification problem
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5) # lr and weight_decay are hyper parameters
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

## Step 7: Train Data

In [ ]:
# train model

net.train()

for epoch in range(15):
  total_correct = 0.0
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_set):
    inputs, labels = inputs.to(device), labels.to(device)
    output = net(inputs)
    output_idx = torch.argmax(output, dim=1)
    total_correct += (labels == output_idx).sum().item()
    optimizer.zero_grad()
    loss = criterion(output, labels)
    running_loss += loss.item() * inputs.size(0)
    loss.backward()
    optimizer.step()
    
  print(f'Epoch: {epoch}  Loss: {running_loss/train_len}   Accuracy:{(total_correct/train_len)*100}%')

print('Finished training')

Step 8: Test Data

In [ ]:
# Test our model

with torch.no_grad(): 
  net.eval()
  total_loss = 0.0
  total_correct = 0.0
  
  for inputs, labels in test_set:
    labels = labels.to(device)
    outputs = net(inputs.to(device))
    loss = criterion(outputs, labels)
    total_loss += loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs, dim=1)
    total_correct += sum(labels==output_idx)
    
  print(f'Accuracy : {(total_correct/test_len)*100}%  Loss: {total_loss/ test_len}')

Step 9: Save trained model